In [38]:
from transformers import pipeline

In [39]:
oracle = pipeline(model="deepset/roberta-base-squad2")
print(oracle(question="What is the capital of Germany?", context="Germany is a country in Europe. The capital is Berlin."))

{'score': 0.9674137234687805, 'start': 47, 'end': 53, 'answer': 'Berlin'}


In [40]:
from SPARQLWrapper import SPARQLWrapper, JSON
def get_context(entity):
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    sparql.setQuery("""
    SELECT ?abstract
        WHERE { 
            <http://dbpedia.org/resource/%s> dbo:abstract ?abstract .
            FILTER (lang(?abstract) = 'en') .
        }
    """ % entity)
    print(sparql.query())
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()

    context = [result["abstract"]["value"] for result in results["results"]["bindings"]]
    return context

In [41]:
oracle = pipeline(model="deepset/roberta-base-squad2")
question="What is the capital of Germany?"
# Use the function to get information on entity
context=get_context("Germany")[0]

{"requestedFormat" : 'xml',
"response (a file-like object, as return by the urllib2.urlopen library call)" : {
	"url" : "https://dbpedia.org/sparql?query=%0A++++SELECT+%3Fabstract%0A++++++++WHERE+%7B+%0A++++++++++++%3Chttp%3A//dbpedia.org/resource/Germany%3E+dbo%3Aabstract+%3Fabstract+.%0A++++++++++++FILTER+%28lang%28%3Fabstract%29+%3D+%27en%27%29+.%0A++++++++%7D%0A++++&format=xml&output=xml&results=xml",
	"code" : "200",
	"headers" : Date: Fri, 01 Dec 2023 19:33:04 GMT
Content-Type: application/sparql-results+xml; charset=UTF-8
Content-Length: 3646
Connection: close
Server: Virtuoso/08.03.3329 (Linux) x86_64-generic-linux-glibc25  VDB
Content-disposition: filename=sparql_2023-12-01_19-29-17Z.xml
Expires: Fri, 08 Dec 2023 19:33:04 GMT
Cache-Control: max-age=604800
Access-Control-Allow-Credentials: true
Access-Control-Allow-Methods: GET, POST, OPTIONS
Access-Control-Allow-Headers: Depth,DNT,X-CustomHeader,Keep-Alive,User-Agent,X-Requested-With,If-Modified-Since,Cache-Control,Content-Typ

In [42]:
print(oracle(question=question, context=context)["answer"])

Berlin
